# 한국어-영어 구어체 번역 seq2seq 모델 설계
 대표적인 encoder-decoder 모델인 `seq2seq`를 사용하여 짧은 corpus에 대한 기계 번역 모델을 설계한다.

In [ ]:
!pip install konlpy
!pip install nltk

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchtext.legacy.data import Field, BucketIterator, TabularDataset

import spacy
import numpy as np
import re

import random
import math
from tqdm import tqdm
import time

from konlpy.tag import Okt

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
random_seed = 2021
torch.manual_seed(random_seed)

## 1. 데이터셋 가져오기
AI hub의 한국어-영어(구어체) 말뭉치를 사용하여 기계 번역을 수행한다.

In [ ]:
dataset = pd.read_excel("/content/drive/MyDrive/datasets/kor_eng_translation.xlsx")
# 'SID'열 제거
dataset = dataset.drop(columns='SID')
dataset.head()

,원문,번역문
0,'Bible Coloring'은 성경의 아름다운 이야기를 체험 할 수 있는 컬러링 ...,Bible Coloring' is a coloring application that...
1,씨티은행에서 일하세요?,Do you work at a City bank?
2,푸리토의 베스트셀러는 해외에서 입소문만으로 4차 완판을 기록하였다.,"PURITO's bestseller, which recorded 4th rough ..."
3,11장에서는 예수님이 이번엔 나사로를 무덤에서 불러내어 죽은 자 가운데서 살리셨습니다.,In Chapter 11 Jesus called Lazarus from the to...
4,"6.5, 7, 8 사이즈가 몇 개나 더 재입고 될지 제게 알려주시면 감사하겠습니다.",I would feel grateful to know how many stocks ...


In [ ]:
len(dataset)

200000

## 2. 데이터 전처리


* 한국어에서는 `정규화`, 영어에서는 `정규화`, `대문자->소문자` 처리를 진행할 예정이다.

In [ ]:
# # 한국어 텍스트 정규화
# remove_kor = [re.sub(r'[^가-힣]', ' ', x) for x in dataset.원문]

# # join
# join = [' '.join(remove_kor[i].split()) for i in range(len(dataset.원문))]

# dataset.원문 = join

# # 영어 텍스트 정규화
# remove_en = [re.sub(r'[^A-Za-z]', ' ', x) for x in dataset.번역문]

# # join
# join = [' '.join(remove_en[i].split()) for i in range(len(dataset.번역문))]

# dataset.번역문 = join

# train, test dataset split
train, val_test = train_test_split(dataset, test_size=0.1)
valid, test = train_test_split(val_test, test_size=0.5)

# train, test dataset 별도 csv 파일로 저장
train.to_csv('/content/drive/MyDrive/datasets/kor_eng_train.csv', index=None)
valid.to_csv('/content/drive/MyDrive/datasets/kor_eng_valid.csv', index=None)
test.to_csv('/content/drive/MyDrive/datasets/kor_eng_test.csv', index=None)

In [ ]:
print("학습 데이터셋 크기: {}".format(len(train)))
print("검증 데이터셋 크기: {}".format(len(valid)))
print("테스트 데이터셋 크기: {}".format(len(test)))

학습 데이터셋 크기: 180000
검증 데이터셋 크기: 10000
테스트 데이터셋 크기: 10000


## 3. DataLoader 생성
`torchtext`를 사용하여 데이터로더를 생성한다. `torchtext`는 아래의 과정을 한 번에 쉽게 할 수 있도록 한다.
* 토크나이징
* vocab 생성
* 토큰의 수치화
* 데이터로더 생성

#### 토큰화 모델 가져오기
영어는 `split`, 한국어는 `Okt`를 사용하여 토큰화를 진행한다.

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# 영어 토큰화 모델
en_tokenizer = lambda x: word_tokenize(x)

# 한국어 토큰화 모델
kor_tokenizer = Okt()

#### 필드 지정
`Field`란, tensor로 표현될 수 있는 텍스트 데이터 타입을 처리한다. 각 토큰을 숫자 인덱스로 `mapping`시켜주는 단어장 객체가 있다. 

In [ ]:
# SRC = source = input
SRC = Field(tokenize=kor_tokenizer.morphs,      # 토큰나이징 모델 지정
            sequential=True,                    # SRC는 순서가 있는, 즉 sequential한 데이터
            init_token='<sos>',                 
            eos_token='<eos>',
            lower=False,                        # 한국어는 대소문자가 없기 때문에 False
            use_vocab=True)                     # 단어장 객체를 사용할지의 여부

# TRG = target = output
TRG = Field(tokenize=en_tokenizer,
            sequential=True,
            init_token='<sos>', 
            eos_token='<eos>', 
            lower=True,                         # 영어는 대소문자가 있기 때문에 True -> 소문자화 
            use_vocab=True,
            is_target=True)

#### 데이터셋 가져오기
이제 위에서 지정한 필드에 기반하여 데이터를 불러온다. 여기서는 `TabularDataset`을 사용하여 데이터셋을 가져온다.


In [ ]:
# fields = (입력, 출력) 
train, valid, test = TabularDataset.splits(path=f'/content/drive/MyDrive/datasets',
                                    train='kor_eng_train.csv',
                                    validation='kor_eng_valid.csv',
                                    test='kor_eng_test.csv',
                                    format='csv',
                                    skip_header=True,
                                    fields=[('source', SRC), ('target', TRG)])

* fields = [("필드이름", 필드객체), ("필드이름", 필드객체)]

결과 확인

In [ ]:
print(vars(train[0]))

{'source': ['저', '는', '일', '을', '많이', '했기', '때문', '에', '쉬어야', '합니다', '.'], 'target': ['i', 'need', 'a', 'break', 'because', 'i', 'worked', 'hard', '.']}


#### Vocabulary 생성
토큰과 인덱스를 1대1 맵핑시키는 단어장을 생성한다. 

In [ ]:
SRC.build_vocab(train)
# TRG.build_vocab(train)

# GloVe를 사용할 경우
TRG.build_vocab(train, vectors="glove.6B.200d")

# FastText를 사용할 경우
# TRG.build_vocab(train, vectors="fasttext.simple.300d")

In [ ]:
print("SRC vocabulary size: {}".format(len(SRC.vocab)))
print("TRG vocabulary size: {}".format(len(TRG.vocab)))

SRC vocabulary size: 69548
TRG vocabulary size: 40694


스페셜 토큰인 `<unk>` `<pad>` `<sos>` `<eos>`의 vocab index를 확인해본다.

In [ ]:
print("==== <unk> <pad> <sos> <eos> token index ====")
print("SRC vocab examples: ", SRC.vocab.stoi['<unk>'], SRC.vocab.stoi['<pad>'], SRC.vocab.stoi['<sos>'], SRC.vocab.stoi['<eos>'])
print("TRG vocab examples: ", TRG.vocab.stoi['<unk>'], TRG.vocab.stoi['<pad>'], TRG.vocab.stoi['<sos>'], TRG.vocab.stoi['<eos>'])

==== <unk> <pad> <sos> <eos> token index ====
SRC vocab examples:  0 1 2 3
TRG vocab examples:  0 1 2 3


#### 단어장 인덱스 확인
토큰화된 단어들에 맵핑된 인덱스를 확인할 수 있다.

In [ ]:
print(SRC.vocab.stoi)
print(TRG.vocab.stoi)

defaultdict(<bound method Vocab._default_unk_index of <torchtext.legacy.vocab.Vocab object at 0x7f83ec9c9c90>>, {'<unk>': 0, '<pad>': 1, '<sos>': 2, '<eos>': 3, '.': 4, '을': 5, '이': 6, '는': 7, '에': 8, '가': 9, '를': 10, '의': 11, '은': 12, '나': 13, '?': 14, '것': 15, ',': 16, '당신': 17, '그': 18, '들': 19, '우리': 20, '수': 21, '에서': 22, '으로': 23, '내': 24, '저': 25, '입니다': 26, '할': 27, '한': 28, '로': 29, '과': 30, '해': 31, '하는': 32, '하고': 33, '합니다': 34, '있습니다': 35, '에게': 36, '와': 37, '도': 38, '적': 39, '사람': 40, '요': 41, '이에요': 42, '생각': 43, '너': 44, '있는': 45, '때': 46, '오늘': 47, '고': 48, '잘': 49, '거': 50, '인': 51, '말': 52, '그녀': 53, '했어요': 54, '때문': 55, '일': 56, '더': 57, '있어요': 58, '못': 59, '친구': 60, '안': 61, '시간': 62, '네': 63, '에는': 64, '했습니다': 65, '하지': 66, '한국': 67, '위해': 68, '서': 69, '제품': 70, '제': 71, '곳': 72, '사용': 73, '해요': 74, '된': 75, '게': 76, '많이': 77, '까지': 78, '중': 79, '가지': 80, '만': 81, '있어': 82, '많은': 83, '그것': 84, '집': 85, '해서': 86, '다': 87, '대해': 88, '난': 89, '확인': 90, '해야': 91, '다른':

#### 소스 데이터 단어 출현 빈도수
`을`, `이`, `는`, `에`와 같은 조사의 빈도수가 크다.

In [ ]:
print(SRC.vocab.freqs)

Counter({'.': 158169, '을': 66274, '이': 60118, '는': 56971, '에': 49850, '가': 36884, '를': 36132, '의': 33883, '은': 33401, '나': 24645, '?': 21518, '것': 21335, ',': 19736, '당신': 17669, '그': 16701, '들': 16206, '우리': 15376, '수': 14125, '에서': 13588, '으로': 13045, '내': 12752, '저': 12625, '입니다': 11610, '할': 11571, '한': 10449, '로': 10336, '과': 9035, '해': 8757, '하는': 8672, '하고': 8575, '합니다': 8292, '있습니다': 8157, '에게': 8059, '와': 7521, '도': 7328, '적': 7162, '사람': 6421, '요': 6379, '이에요': 5945, '생각': 5720, '너': 5453, '있는': 5157, '때': 5111, '오늘': 5033, '고': 4764, '잘': 4609, '거': 4600, '인': 4503, '말': 4473, '그녀': 4378, '했어요': 4331, '일': 4290, '때문': 4290, '더': 4236, '있어요': 4191, '못': 4101, '친구': 3907, '안': 3715, '시간': 3591, '네': 3545, '에는': 3532, '했습니다': 3507, '하지': 3409, '한국': 3348, '위해': 3265, '서': 3218, '제품': 3131, '제': 3087, '곳': 3055, '사용': 3013, '해요': 3010, '된': 2974, '게': 2953, '많이': 2888, '까지': 2833, '중': 2813, '가지': 2799, '만': 2794, '있어': 2792, '많은': 2740, '그것': 2739, '집': 2719, '해서': 2673, '다': 2

#### 타겟 데이터 단어 출현 빈도수
`the`, `i`, `to`, `a` 등의 단어의 빈도수가 크다.

In [ ]:
print(TRG.vocab.freqs)

Counter({'.': 157960, 'the': 98685, 'i': 82670, 'to': 63335, ',': 49023, 'a': 48058, 'you': 45351, 'is': 36169, 'and': 34518, 'of': 32763, 'in': 28334, 'it': 28329, '?': 21576, 'for': 19857, 'that': 19334, 'we': 18198, 'my': 17532, 'this': 16229, 'have': 15605, 'with': 14427, 'are': 14009, 'do': 13663, "'s": 13280, "n't": 12610, 'on': 12519, 'be': 12474, 'will': 12386, 'can': 12110, 'me': 11243, 'your': 10916, 'was': 10732, 'if': 10239, 'as': 9967, 'not': 9884, 'at': 8701, 'there': 8650, 'so': 8438, 'like': 7925, 'when': 7244, 'from': 7205, 'because': 6839, 'want': 6754, 'but': 6658, 'by': 6649, "'m": 6614, 'please': 6600, 'time': 6538, 'about': 6406, 'he': 6372, 'an': 5907, 'what': 5688, 'go': 5511, 'am': 5428, 'has': 4864, 'think': 4649, 'get': 4620, 'know': 4617, 'after': 4605, 'would': 4600, 'our': 4385, 'they': 4307, 'today': 4191, 'all': 4140, 'people': 4050, 'how': 4003, 'who': 3951, 'she': 3945, 'did': 3890, 'good': 3865, 'one': 3759, 'more': 3601, 'or': 3522, 'also': 3480, 'wo

#### 데이터로더 만들기
`BucketIterator`를 사용하여 데이터로더를 만든다. 
* 유사한 길이의 텍스트 sequence를 일괄 처리하여 그룹화한다.
* 매 배치마다 최대 길이에 따라 알아서 padding을 해주게 되는데, 이 때 유사한 길이의 sequence끼리 배치로 묶이면 너무 많은 공간을 `<pad>`로 채워줄 필요가 없다.

In [ ]:
batch_size = 256

train_iter = BucketIterator(train,
                            batch_size=batch_size,
                            sort_key=lambda x: len(x.source),
                            shuffle=True,
                            sort_within_batch=True,
                            repeat=False,
                            sort=False,
                            device=device)

valid_iter = BucketIterator(valid,
                           batch_size=batch_size,
                           sort_key=lambda x: len(x.source),
                           shuffle=False,
                           sort_within_batch=True,
                           repeat=False,
                           sort=False,
                           device=device)

test_iter = BucketIterator(test,
                           batch_size=batch_size,
                           sort_key=lambda x: len(x.source),
                           shuffle=False,
                           sort_within_batch=True,
                           repeat=False,
                           sort=False,
                           device=device)

In [ ]:
# iterator 길이
print("Train iterator 길이: {}".format(len(train_iter)))
print("Validation iterator 길이: {}".format(len(valid_iter)))
print("Validation iterator 길이: {}".format(len(test_iter)))


Train iterator 길이: 704
Validation iterator 길이: 40
Validation iterator 길이: 40


### 각 배치의 데이터의 길이
비슷한 길이의 데이터끼리 배치로 묶임으로써 `<pad>`토큰이 사용되는 공간을 최소화한다.

In [ ]:
epochs = 1

for epoch in range(epochs):

  train_iter.create_batches()

  for sample_id, batch in enumerate(train_iter.batches):
      print('Batch examples lengths: %s'.ljust(20) % str([len(example.source) for example in batch]))
      
      if sample_id == 10:
          break

Batch examples lengths: [16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,

## 4. 모델 설계
### 4-1. 임베딩 테이블 만들기
먼저 앞서 만든 SRC, TRG vocab 사전을 사용하여 임베딩 테이블을 만든다.
* `num_embedding`: 단어 집합의 크기
* `embedding_dim`: 임베딩 벡터 차원
* `padding_idx`: padding을 할 토큰의 인덱스를 알려줌.

In [ ]:
enc_embedding_layer = nn.Embedding(num_embeddings=len(SRC.vocab),
                                  embedding_dim=200,
                                  padding_idx=SRC.vocab.stoi[SRC.pad_token])

# Baseline
dec_embedding_layer = nn.Embedding(num_embeddings=len(TRG.vocab),
                                   embedding_dim=200,
                                   padding_idx=TRG.vocab.stoi[TRG.pad_token])

# Glove, FastText 사용할 경우
# dec_embedding_layer= nn.Embedding.from_pretrained(TRG.vocab.vectors, freeze=False)

* Encoder의 Embedding layer는 64,264개의 단어, Embedding 차원은 200, padding의 인덱스는 1이다.
* Decoder의 Embedding layer는 34,331개의 단어, Embedding 차원은 200, padding의 인덱스는 1이다.

In [ ]:
enc_embedding_layer, dec_embedding_layer

(Embedding(69548, 200, padding_idx=1), Embedding(40694, 200, padding_idx=1))

In [ ]:
print("==== Encoder Embedding layer ====")
print("size: ", len(enc_embedding_layer.weight))
print(enc_embedding_layer.weight)
print("\n==== Decoder Embedding layer ====")
print("size: ", len(dec_embedding_layer.weight))
print(dec_embedding_layer.weight)

==== Encoder Embedding layer ====
size:  69548
Parameter containing:
tensor([[-0.1452,  0.9747,  0.6023,  ...,  0.4883,  2.0643, -0.8933],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.4100,  0.7935, -0.0261,  ...,  1.5027,  0.0366,  1.9288],
        ...,
        [-0.2802, -0.1767,  1.1068,  ..., -1.1210, -0.2502,  0.3185],
        [-0.5900, -0.1897,  0.6587,  ...,  0.1333,  0.1009, -0.9851],
        [-0.2576, -0.2537,  0.0385,  ..., -0.1219,  1.2593, -0.4995]],
       requires_grad=True)

==== Decoder Embedding layer ====
size:  40694
Parameter containing:
tensor([[ 1.4774,  0.0616,  1.0172,  ..., -2.1532, -0.6980,  0.3839],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-1.0733, -0.1173, -0.4494,  ...,  0.8966,  0.8755, -0.3445],
        ...,
        [-0.6186, -0.0149, -0.2608,  ..., -0.3136,  1.4943,  1.0721],
        [ 0.7302, -0.7262, -1.2138,  ...,  0.5758,  0.0681,  0.4703],
        [ 0.9374,  0.3579, -0.6188,  

단어 집합의 크기를 갖는 임베딩 테이블이 생성된 것을 볼 수 있다.

### 4-2. 모델 설계
* Encoder
* Decoder
* Seq2Seq

#### Encoder
임의의 길이의 sequence를 고정 길이 벡터로 변환한다.

In [ ]:
class Encoder(nn.Module):
    def __init__(self, emb_dim, hidden_dim, emb_table, n_layers, dropout):
        super().__init__()

        # Hidden layer dimension
        self.hidden_dim = hidden_dim

        # Embedding Table
        self.embedding = emb_table

        # Layer 개수
        self.n_layers = n_layers

        # dropout
        self.dropout = nn.Dropout(p=dropout)

        # RNN 정의
        self.rnn = nn.LSTM(emb_dim, hidden_dim, n_layers, dropout=dropout)


    def forward(self, source):
        out = self.embedding(source)
        out = self.dropout(out)
        # hidden state, cell state
        # hidden state는 context vector라고도 불린다.
        outputs, (hidden, cell) = self.rnn(out)

        return hidden, cell

* 입력 sequence를 LSTM을 통해 hidden state를 전달한다.
* 마지막 단계의 hidden state를 Decoder로 전달한다.
* 이 마지막 hidden state는 Context Vector라고 한다.
* Context Vector는 입력 sequence의 정보를 하나의 고정된 크기 벡터 표현으로 압축한 것.
* Encoding: 임의의 길이의 sequence를 고정 길이 벡터로 변환하는 작업
* 문제점: 아무리 입력 Sequence의 길이가 길어도 항상 같은 길이의 벡터로 변환된다. 따라서 필요한 정보가 벡터에 다 담기지 못하게 된다.

#### Decoder

In [ ]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hidden_dim, emb_table, n_layers, dropout):
        super().__init__()

        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        self.embedding = emb_table
        self.n_layers = n_layers

        self.rnn = nn.LSTM(emb_dim, hidden_dim, n_layers, dropout=dropout)
        # Affine layer
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, input, hidden, cell):
        input = input.unsqueeze(0)              # axis=0에 1인 차원 생성
        out = self.embedding(input)
        out = self.dropout(out)
        output, (hidden, cell) = self.rnn(out, (hidden, cell))
        prediction = self.fc(output.squeeze(0))

        return prediction, hidden, cell

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, source, target, teacher_forcing_ratio=0.5):
        target_len = target.shape[0]
        batch_size = target.shape[1]
        target_vocab_size = self.decoder.output_dim

        # decoder 결과를 저장할 텐서
        outputs = torch.zeros(target_len, batch_size, target_vocab_size).to(self.device)

        # Encoder의 마지막 hidden state가 Decoder의 initial hidden state 상태로 쓰임
        hidden, cell = self.encoder(source)

        # Decoder에 들어갈 첫 input은 <sos> 토큰
        input = target[0, :]
        
        # target length만큼 반복
        # range(0, target_len)이 아니라 range(1, target_len)인 이유: 0번째 target은 항상 <sos>라서 그에 대한 output도 항상 0
        for i in range(1, target_len):
            output, hidden, cell = self.decoder(input, hidden, cell)
            outputs[i] = output
            

            # random.random(): [0,1] 랜덤 숫자
            # 랜덤 숫자가 teacher_forcing_ratio보다 작으면 True니까 teacher_forcing=1
            teacher_force = 1 if random.random() < teacher_forcing_ratio else 0

            # 확률 가장 높게 예측한 토큰
            top_token = output.argmax(1) #F.softmax(output, dim=1)

            # teacher_force = 1: True면 target[i]를, 아니면 확률 가장 높게 예측한 토큰을 input으로 사용
            input = target[i] if teacher_force else top_token
        return outputs

In [ ]:
output_dim = len(TRG.vocab)

# Encodr embedding dim
enc_emb_dim = 200               # FastText 사용할 경우 300으로 변경
# Decoder embedding dim
dec_emb_dim = 200               # FastText 사용할 경우 300으로 변경

hidden_dim = 512
n_layers = 2

dropout = 0.5

# parameter: emb_dim, hidden_dim, emb_table, n_layers, dropout
encoder = Encoder(enc_emb_dim, hidden_dim, enc_embedding_layer, n_layers, dropout)
# parameter: output_dim, emb_dim, hidden_dim, emb_table, n_layers, dropout
decoder = Decoder(output_dim, dec_emb_dim, hidden_dim, dec_embedding_layer, n_layers, dropout)

model = Seq2Seq(encoder, decoder, device).to(device)

In [ ]:
print(encoder)
print(decoder)
print(model)

Encoder(
  (embedding): Embedding(69548, 200, padding_idx=1)
  (dropout): Dropout(p=0.5, inplace=False)
  (rnn): LSTM(200, 512, num_layers=2, dropout=0.5)
)
Decoder(
  (embedding): Embedding(40694, 200, padding_idx=1)
  (rnn): LSTM(200, 512, num_layers=2, dropout=0.5)
  (fc): Linear(in_features=512, out_features=40694, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)
Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(69548, 200, padding_idx=1)
    (dropout): Dropout(p=0.5, inplace=False)
    (rnn): LSTM(200, 512, num_layers=2, dropout=0.5)
  )
  (decoder): Decoder(
    (embedding): Embedding(40694, 200, padding_idx=1)
    (rnn): LSTM(200, 512, num_layers=2, dropout=0.5)
    (fc): Linear(in_features=512, out_features=40694, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)


### 가중치 초기화 메소드

In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)

model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(69548, 200, padding_idx=1)
    (dropout): Dropout(p=0.5, inplace=False)
    (rnn): LSTM(200, 512, num_layers=2, dropout=0.5)
  )
  (decoder): Decoder(
    (embedding): Embedding(40694, 200, padding_idx=1)
    (rnn): LSTM(200, 512, num_layers=2, dropout=0.5)
    (fc): Linear(in_features=512, out_features=40694, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [ ]:
# default lr=0.001
optimizer = optim.Adam(model.parameters())

trg_pad_idx = 1

criterion = nn.CrossEntropyLoss(ignore_index=trg_pad_idx)

### 학습 메소드

In [ ]:
def train(model, iterator, optimizer, criterion, clip):
    model.train()
    epoch_loss = 0
    for i, batch in enumerate(tqdm(iterator, desc="Training")):
        time.sleep(0.1)
        # if i % 100 == 0:
        #     print("{}/{}".format(i, len(iterator)))
        
        source = batch.source
        target = batch.target

        optimizer.zero_grad()

        output = model(source, target)
        output_dim = output.shape[-1]

        # loss 함수는 2d input으로만 계산 가능
        output = output[1:].view(-1, output_dim)
        target = target[1:].view(-1)

        loss = criterion(output, target)

        loss.backward()

        # Gradient Exploding을 막기 위한 clip
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()
        epoch_loss += loss.item()

    return epoch_loss / len(iterator)

### 평가 메소드

In [ ]:
def evaluate(model, iterator, criterion):
    model.eval()
    epoch_loss = 0

    with torch.no_grad():
        for i, batch in enumerate(tqdm(iterator, desc="Evaluating")):
            time.sleep(0.1)
            source = batch.source
            target = batch.target

            # teacher_forcing_ratio = 0 : 아무것도 알려주면 안됨
            output = model(source, target, 0)

            # output = [target lenght, batch size, output dim]
            output_dim = output.shape[-1]

            output = output[1:].view(-1, output_dim)
            target = target[1:].view(-1)

            loss = criterion(output, target)

            epoch_loss += loss.item()

        return epoch_loss / len(iterator)

In [ ]:
def epoch_time(start_time, end_time):
    total_time = end_time - start_time
    min = int(total_time / 60)
    sec = int(total_time - (min * 60))
    return min, sec

## 5. Training

In [ ]:
EPOCHS = 20
CLIP = 1
best_valid_loss = float('inf')

for epoch in range(EPOCHS):
    print("[Epoch {}]".format(epoch+1))
    start_time = time.time()

    train_loss = train(model, train_iter, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iter, criterion)
    end_time = time.time()

    min, sec = epoch_time(start_time, end_time)
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), '/content/drive/MyDrive/Colab Notebooks/portfolio/model/kor_en_translate_seq2seq.pt')

    print(f"Time: {min}m {sec}s")
    print(f"Train Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}")
    print(f"Validation Loss: {valid_loss:.3f} | Validation PPL: {math.exp(valid_loss):7.3f}\n")

[Epoch 1]


Evaluating: 100%|██████████| 40/40 [00:23<00:00,  1.72it/s]


Time: 10m 44s
Train Loss: 5.537 | Train PPL: 254.028
Validation Loss: 5.631 | Validation PPL: 278.882

[Epoch 2]


Evaluating: 100%|██████████| 40/40 [00:23<00:00,  1.72it/s]


Time: 10m 42s
Train Loss: 4.806 | Train PPL: 122.241
Validation Loss: 5.235 | Validation PPL: 187.786

[Epoch 3]


...

Time: 10m 45s
Train Loss: 2.423 | Train PPL:  11.279
Validation Loss: 4.467 | Validation PPL:  87.075

[Epoch 19]


Evaluating: 100%|██████████| 40/40 [00:23<00:00,  1.71it/s]


Time: 10m 43s
Train Loss: 2.376 | Train PPL:  10.764
Validation Loss: 4.499 | Validation PPL:  89.927

[Epoch 20]


Evaluating: 100%|██████████| 40/40 [00:23<00:00,  1.71it/s]

Time: 10m 45s
Train Loss: 2.338 | Train PPL:  10.358
Validation Loss: 4.532 | Validation PPL:  92.914



## 6. 결과 확인

In [ ]:
# 출처: https://github.com/ndb796/Deep-Learning-Paper-Review-and-Practice/blob/master/code_practices/Sequence_to_Sequence_with_LSTM_Tutorial.ipynb
# 번역(translation) 함수
def translate_sentence(sentence, src_field, trg_field, model, device, max_len=50):
    model.eval() # 평가 모드

    if isinstance(sentence, str):
        nlp = spacy.load('en')
        tokens = [token.text.lower() for token in nlp(sentence)]
    else:
        tokens = [token.lower() for token in sentence]

    # 처음에 <sos> 토큰, 마지막에 <eos> 토큰 붙이기
    tokens = [src_field.init_token] + tokens + [src_field.eos_token]
    # print(f"전체 소스 토큰: {tokens}")

    src_indexes = [src_field.vocab.stoi[token] for token in tokens]
    # print(f"소스 문장 인덱스: {src_indexes}")

    src_tensor = torch.LongTensor(src_indexes).unsqueeze(1).to(device)

    # 인코더(endocer)에 소스 문장을 넣어 문맥 벡터(context vector) 계산
    with torch.no_grad():
        hidden, cell = model.encoder(src_tensor)

    # 처음에는 <sos> 토큰 하나만 가지고 있도록 하기
    trg_indexes = [trg_field.vocab.stoi[trg_field.init_token]]

    for i in range(max_len):
        # 이전에 출력한 단어가 현재 단어로 입력될 수 있도록
        trg_tensor = torch.LongTensor([trg_indexes[-1]]).to(device)

        with torch.no_grad():
            output, hidden, cell = model.decoder(trg_tensor, hidden, cell)

        pred_token = output.argmax(1).item()
        trg_indexes.append(pred_token) # 출력 문장에 더하기

        # <eos>를 만나는 순간 끝
        if pred_token == trg_field.vocab.stoi[trg_field.eos_token]:
            break

    # 각 출력 단어 인덱스를 실제 단어로 변환
    trg_tokens = [trg_field.vocab.itos[i] for i in trg_indexes]

    # 첫 번째 <sos>는 제외하고 출력 문장 반환
    return trg_tokens[1:]

#### 랜덤으로 5개의 문장을 골라 번역 성능을 확인한다.

In [ ]:
example_idx = [random.randint(0, 10000) for i in range(5)]
for idx in example_idx:
    src = vars(test.examples[idx])['source']
    trg = vars(test.examples[idx])['target']

    print("\n============ Test index {} ==============".format(idx))
    print(f'원  문: {src}')
    print(f'번역문: {trg}')
    print("모델 출력 결과:", " ".join(translate_sentence(src, SRC, TRG, model, device)))


============ Test index 1415 ==============
원  문: ['너', '에게', '메시지', '를', '보냈지만', ',', '넌', '답장', '이', '없었어', '.']
번역문: ['i', 'sent', 'you', 'a', 'message', ',', 'but', 'you', 'did', 'not', 'reply', '.']
모델 출력 결과: i sent you a message but i did n't reply . <eos>

============ Test index 4997 ==============
원  문: ['저', '는', '그것', '이', '사실', '이라는', '것', '을', '알', '고', '있어요', '.']
번역문: ['i', 'know', 'it', 'is', 'a', 'fact', '.']
모델 출력 결과: i know that it 's true . <eos>

============ Test index 6637 ==============
원  문: ['저', '는', '간단한', '아침', '을', '먹습니다', '.']
번역문: ['i', 'have', 'a', 'light', 'breakfast', '.']
모델 출력 결과: i eat a simple breakfast . <eos>

============ Test index 4404 ==============
원  문: ['당신', '의', '샤우트', '창법', '강의', '는', '아주', '특별했습니다', '.']
번역문: ['your', 'lecture', 'about', 'the', 'shout', 'vocal', 'technique', 'was', 'very', 'special', '.']
모델 출력 결과: your best lecture was also good for you . <eos>

============ Test index 3220 ==============
원  문: ['그녀', '는', '자기', '오빠